In [1]:
import pandas as pd
from gazpacho import get, Soup # Import gazpacho

url = "https://tax.alaska.gov/programs/oil/production/ans.aspx?" # Set the primary url value as a "string"

html = get(url) # Get the html from the url
soup = Soup(html) # Pass html into soup
list = soup.find('option')[1:] # Find all the 'option' html tags and add to list
length = len(list) # Number of records in the list (month/year)

urls = [] # Create an empty list for all the urls

# 1. Scrape the dropdown selection menu for website urls containing html data tables

for i in range(length): # Loop through all the html tags for 'options' for the length of the list
    options = list[i].attrs['value'] # Create variable and set it to the 'value' in the options list
#     print(url + options) # Print a list of urls to be scraped
    urls.append(url + options) # and add the value to the urls list
    
url_count = len(urls) # Number of urls in the list

# 2. Scrape the website urls containing html data tables

for url in urls:
    
    file_df = pd.read_csv('Resources/alaska_oil_production.csv') # Read previously scraped data file into Pandas
    file_df['Date'] = pd.to_datetime(file_df['Date'], errors='coerce') # Convert column to datetime datatype
    
    scraped_df = pd.read_html(url)[6] # Read the 7th data table from each url
    scraped_df = pd.DataFrame(scraped_df) # Set scraped_df as a dataframe

    scraped_df.columns = file_df.columns # Set the columns name from the file equal to the scraped_df

    scraped_df['Date'] = pd.to_datetime(scraped_df['Date'], errors='coerce') # Convert the 'Date' column to datetime datatype
    scraped_df = scraped_df.dropna(axis=0, how='any', subset=['Date']) # Drop any rows with NaN in 'Date'

    merged_df = file_df.append(scraped_df) # Merge the scraped_df to the file_df
    merged_df = merged_df.sort_values(by='Date', ascending=False) # Sort by 'Date' in descending order
    merged_df['duplicates'] = merged_df.duplicated('Date', keep='last') # Create a column to assign duplicates T/F
    merged_df = merged_df[merged_df['duplicates'] == False] # Save df with duplicates equal to False only
    merged_df = merged_df.drop(columns=['duplicates'])# Delete the 'duplicates' column if dupe_count = 0
    merged_df = merged_df.reset_index(drop=True) # Reset the merged dataframe index
    merged_df.to_csv('Resources/alaska_oil_production.csv', mode='w', index=False) # Write merged dataframe to existing csv

In [2]:
merged_df.head()

,Date,Prudhoe Bay,Kuparuk,Endicott,Lisburne,Alpine,ANS,Inventories,Milne Point,Northstar,Cook Inlet,Alaska,PS# Temperature
0,2021-12-09,323228.0,104546.0,0.0,19993.0,47634.0,495400.0,3395162.0,NaN,NaN,NaN,NaN,NaN
1,2021-12-08,322793.0,105710.0,0.0,16867.0,48747.0,494117.0,4468348.0,NaN,NaN,NaN,NaN,NaN
2,2021-12-07,318355.0,105475.0,0.0,10630.0,46872.0,481333.0,3995082.0,NaN,NaN,NaN,NaN,NaN
3,2021-12-06,309144.0,107631.0,0.0,20517.0,48515.0,485807.0,3508575.0,NaN,NaN,NaN,NaN,NaN
4,2021-12-05,319178.0,109334.0,0.0,21926.0,49369.0,499807.0,3349649.0,NaN,NaN,NaN,NaN,NaN
